In [3]:
import pandas as pd
import numpy as np
from yaml import safe_load
import os 
from tqdm  import tqdm
import warnings
import yaml
warnings.filterwarnings("ignore")

In [4]:
files_name = []
dir_path = './data'

for file in os.listdir(dir_path):
    # Get the full file path
    full_path = os.path.join(dir_path, file)
    # Check if the file is not a .txt file since there is one README.txt in the folder
    if not file.endswith('.txt'):
        # Add the file path to the list
        files_name.append(full_path)

In [5]:
final_df=pd.DataFrame()
counter=1
for i in tqdm(files_name):
    with open(i,'r') as f:
        data = yaml.safe_load(f)
        if isinstance(data, list) or isinstance(data, dict):
            df = pd.json_normalize(data)
        else:
            raise ValueError(f"File {i} does not contain a list or dict")
        df['match_id']=counter
        final_df=pd.concat([final_df,df],ignore_index=True)
        counter+=1
final_df

  4%|█▌                                      | 152/3857 [00:14<05:57, 10.35it/s]


KeyboardInterrupt: 

In [ ]:
cl = final_df.columns

In [ ]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.supersubs.South Africa',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

In [ ]:
selected_columns = [
    "innings", "info.dates", "info.gender", "info.match_type", 
    "info.outcome.winner", "info.overs", "info.player_of_match", 
    "info.teams", "info.toss.decision", "info.toss.winner", 
    "info.umpires", "info.venue", "match_id", "info.city"
]

final_df = final_df[selected_columns]


In [ ]:
final_df

In [ ]:
final_df.to_csv("final_df",index = False)

In [ ]:
final_df["info.gender"].value_counts()

In [ ]:
final_df = final_df[final_df["info.gender"] == "male"]

In [ ]:
final_df.drop(columns=['info.gender'],inplace=True)


In [ ]:
final_df["info.match_type"].value_counts()

In [ ]:
final_df["info.overs"].value_counts()

In [ ]:
final_df = final_df[final_df["info.overs"] == 20]

In [ ]:
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)
final_df

In [ ]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))

In [ ]:
matches = pickle.load(open('dataset_level1.pkl','rb'))
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

In [ ]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
   
delivery_df = pd.concat(delivery_dataframes, ignore_index=True)

In [ ]:
delivery_df

In [ ]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [ ]:

delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)

In [ ]:
delivery_df

In [ ]:
delivery_df.drop(columns=['teams'],inplace=True)

In [ ]:
delivery_df['batting_team'].unique()


In [ ]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [ ]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [ ]:
delivery_df

In [ ]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [ ]:
output

In [ ]:
pickle.dump(output,open('dataset_level2.pkl','wb'))

In [ ]:
output.to_csv("final_df",index = False)